In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
directory_path = "/home/kwat/Downloads/prism/"

kraft.path.path(directory_path)

In [ ]:
for url in (
    "https://ndownloader.figshare.com/files/20237712",
    "https://ndownloader.figshare.com/files/20237760",
    "https://ndownloader.figshare.com/files/20237739",
):

    print(kraft.internet.download(url, directory_path,))

## Compound dose (1 and 2)

In [ ]:
def name_compound_dose(id_dose, id_to_compound):

    id_, dose = id_dose.split("::")[:2]

    return "{}_{:.2f}".format(id_to_compound[id_], float(dose))

In [ ]:
for version in (
    "primary",
    "secondary",
):

    print("=" * 80)

    print(version)

    id_to_compound = (
        pd.read_csv(
            "{}/{}-screen-replicate-treatment-info.csv".format(directory_path, version),
            index_col=1,
        )
        .loc[:, "name"]
        .to_dict()
    )

    compound_dose_x_cell_line = pd.read_csv(
        "{}/{}-screen-replicate-collapsed-logfold-change.csv".format(
            directory_path, version
        ),
        index_col=0,
    ).T

    compound_dose_x_cell_line.index = (
        name_compound_dose(compound_dose, id_to_compound)
        for compound_dose in compound_dose_x_cell_line.index.to_numpy()
    )

    print(compound_dose_x_cell_line.shape)

    compound_dose_x_cell_line = compound_dose_x_cell_line.groupby(level=0).median()

    print(compound_dose_x_cell_line.shape)

    compound_dose_x_cell_line.columns = kraft.cell_line.rename(
        compound_dose_x_cell_line.columns.to_numpy()
    )

    compound_dose_x_cell_line.index.name = "Compound Dose"

    compound_dose_x_cell_line.columns.name = "Cell Line"

    kraft.dataframe.error_axes(compound_dose_x_cell_line)

    compound_dose_x_cell_line.to_csv(
        "{}/{}.compound_dose_x_cell_line.tsv".format(
            directory_path, {"primary": 1, "secondary": 2}[version]
        )
    )

    print(compound_dose_x_cell_line.head(n=2))

## Response curve (2)

In [ ]:
cell_lines, scores, compounds = (
    pd.read_csv(
        "{}/secondary-screen-dose-response-curve-parameters.csv".format(directory_path),
        usecols=(1, 8, 11),
    )
    .to_numpy()
    .T
)

cell_lines = np.asarray(kraft.cell_line.rename(cell_lines))

scores = scores.astype(float)

In [ ]:
compound_x_cell_line = kraft.dataframe.pivot(
    compounds,
    cell_lines,
    scores,
    function=min,
    axis_0_name="Compound",
    axis_1_name="Cell Line",
)

compound_x_cell_line.to_csv(
    "{}/2.compound_x_cell_line.tsv".format(directory_path), sep="\t"
)

compound_x_cell_line